In [35]:
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest

In [43]:
# 获取 mnist 数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/data/tdr/deeplearning/data", one_hot=True)

Extracting /data/tdr/deeplearning/data/train-images-idx3-ubyte.gz
Extracting /data/tdr/deeplearning/data/train-labels-idx1-ubyte.gz
Extracting /data/tdr/deeplearning/data/t10k-images-idx3-ubyte.gz
Extracting /data/tdr/deeplearning/data/t10k-labels-idx1-ubyte.gz


In [44]:
tiShape = mnist.train.images.shape
tlShape = mnist.train.labels.shape
print(tiShape,tlShape)

(55000, 784) (55000, 10)


In [45]:
num_steps = 500
batch_size = 1024
num_classes = 10
num_features = 784
num_trees = 10
max_nodes = 1000

x = tf.placeholder(tf.float32, [None, num_features])
y = tf.placeholder(tf.float32, [None])

hparams = tensor_forest.ForestHParams(num_classes=num_classes, num_features=num_features, num_trees=num_trees, max_nodes=max_nodes).fill()

In [46]:
"""
forest_graph = tensor_forest.RandomForestGraphs(hparams)
train_op = forest_graph.training_graph(x, y)
loss_op = forest_graph.training_loss(x, y)

infer_op = forest_graph.inference_graph(x)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init_vars = tf.global_variables_initializer()
"""
# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)
# Get training graph and loss
train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)

# Measure the accuracy
infer_op = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 10, 'max_nodes': 1000, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 784, 'max_fertile_nodes': 500, 'split_after_samples': 250, 'min_split_samples': 5, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'num_classes': 10, 'num_features': 784, 'bagged_num_features': 784, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 11, 'split_initializations_per_input': 3, 'base_random_seed': 0}


ValueError: Variable device_dummy_0 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/contrib/tensor_forest/python/tensor_forest.py", line 288, in __init__
    name='device_dummy_%d' % i, shape=0))
  File "/home/tdr/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/contrib/tensor_forest/python/tensor_forest.py", line 318, in __init__
    tree_variables_class=tree_variables_class)
  File "<ipython-input-22-ac2d3ee2e92a>", line 1, in <module>
    forest_graph = tensor_forest.RandomForestGraphs(hparams)


In [47]:
sess = tf.Session()
sess.run(init_vars)
for i in range(1, num_steps + 1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    _, l = sess.run([train_op, loss_op], feed_dict={x: batch_x, y: batch_y})
    if i % 50 == 0 or i == 1:
        acc = sess.run(accuracy_op, feed_dict={x: batch_x, y: batch_y})
        print("step %d, loss: %f, accuracy: %f" % (i, l, acc))
test_x, test_y = mnist.test.images, mnist.test.labels
print("test accuracy: ", sess.run(accuracy_op, feed_dict={x: test_x, y: test_y}))

ValueError: Cannot feed value of shape (1024, 10) for Tensor 'Placeholder_9:0', which has shape '(?,)'